- this file is for developing a script to reformat all of the individual csv files generated for each descriptive construction into one corpus file
- the corpus queries are tab-separated and span three columns, so that has to be tidied up
- then all sentences are combined into one list, which can then be exported
- V1 uses exported files from ENCOW16A-NANO, which might be sufficient? 

In [30]:
import os

In [33]:
def get_sents_from_csv(subdir, filename):
    """
    Reformats the tab-separated outputs of SeaCOW queries for each descriptive construction into a list of sentences.
    
    Args:
        subdir: a string with the subdirectory containing the CSV file.
        filename: a string ending in '.csv'; the file to read in.
    Returns:
        A list of sentences as strings from the given concordance file.
    """
    
    # Initialise empty list to fill as we go through the concordance file.
    sent_list = []
    
    path = str(subdir + '/' + filename)

    # Open desired file and go through it line by line.
    with open( path , encoding='utf-8') as file:
        for line in file:

            # We only care about the actual concordance, so, the lines in output file not beginning with # or the header 'doc'.
            if line[0] != "#" and line[0:3] != "doc":

                # Remove newlines and split by tabs (since the output is actually tab-separated), saving all but the first
                # element of the resulting list (the original URL of the sentence) as a string to the concordance's list.
                split_by_tab = line.strip('\n').split('\t')
                sent_as_str = " ".join( split_by_tab[1:] )                
                sent_list.append(sent_as_str)
        
    return sent_list

In [39]:
SUBDIR = 'encow16a-nano'

conc_files = [file for file in os.listdir(SUBDIR) if file[-3:] == 'csv']
conc_files
# smell_like = get_sents_from_csv(SUBDIR, 'smell_like.csv')

['be_another_word_for.csv',
 'be_det.csv',
 'be_not_det.csv',
 'be_part_of.csv',
 'be_related_to.csv',
 'be_something_that_you.csv',
 'be_usually_used.csv',
 'can_be_found.csv',
 'feel_like.csv',
 'look_like.csv',
 'means.csv',
 'smell_like.csv',
 'sound_like.csv']